# Tarea 4: Personalización

El objetivo de la tarea 4 es segmentar los 10.000 emails de la recomendación de productos (tarea 3) en 4 o 5 creatividades diferentes en función del perfil de los  clientes, para comunicarnos con ellos de una forma más efectiva.

Para alcanzar este objetivo aplicaremos tareas de aprendizaje no supervisado de Machine Learning. En concreto, aplicaremos el algoritmo de K-Means para particionar un conjunto de 10.000 clientes en 4 o 5 clusters. Cumpliendo las siguientes condiciones:

1) Cada cliente será asignado a un solo cluster.

2) Todos los clusters tendrán al menos un cliente asignado.

3) Los clientes pertenecerán a un solo cluster, no habrán solapes.

## Tabla de contenido <a class="anchor" id="0"></a>

1. [Importación de librerías](#origin) <br> 
2. [Importación de los datasets](#01) <br> 
3. [Construcción del dataset ](#02) <br>
3. [Incorporación, generación y transformación de atributos](#03) <br>
3. [Limpieza de atributos](#04) <br>
3. [Tratamiento de nulos](#05) <br>
3. [Selección de los datos](#06) <br>
3. [Modelling](#07) <br>
3. [Elbow curve](#08) <br>
3. [Descripción de los clusters](#09) <br>
10. [Conclusiones](#10) <br> 





## Importación de librerías <a class="anchor" id="origin"></a>
[Tabla de Contenidos](#0)

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import os

import warnings
warnings.filterwarnings("ignore")

# time calculation to track some processes
import time

# python core library for machine learning and data science
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.cluster import KMeans

!pip install xlrd==1.2.0

import xlrd

import pickle

## Importación de los datasets <a class="anchor" id="01"></a>
[Tabla de Contenidos](#0)

In [ ]:
PATH_DATA = "../input/easymoneygrupo5/"

In [ ]:
comercial = pd.read_csv(PATH_DATA+'commercial_activity_df.csv', encoding='utf-8')
comercial.drop(columns=['Unnamed: 0'], inplace=True)

productos = pd.read_csv(PATH_DATA+'products_df.csv', encoding='utf-8')
productos.drop(columns=['Unnamed: 0'], inplace=True)

socio = pd.read_csv(PATH_DATA+'sociodemographic_df.csv', encoding='utf-8')
socio.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
# para aligerar el tamaño de los ficheros se cambia el tipo de datos a int8
for i in productos.columns:
        if (productos[i].dtype=="int64" and [i]!=["pk_cid"]):
            productos[i]=productos[i].astype("int8")
        else:            
            productos[i]=productos[i]
            
comercial["active_customer"]=comercial["active_customer"].astype('int8')
socio["age"]=socio["age"].astype('int8')

In [ ]:
recomendaciones_finales = pd.read_csv(PATH_DATA+'recomendaciones_finales.csv', encoding='utf-8')
recomendaciones_finales.drop(columns=['Unnamed: 0'], inplace=True)

### Incorporación de datos de fuentes externas
Añadimos el dataset pibpc que tiene datos sobre el PIB per cápita y la población por provincias en España (fuente INE España) y el PIB per cápita de los países extranjeros (fuente: World Bank), para el año 2018. Población de las capitales de los países extranjeros (fuente: World Bank).

Además, tenemos una variable binaria 0 y 1 que indica si en la provincia o en el extranjero se habla o no otro idioma (0 castellano, 1 otro idioma).

In [ ]:
pibpc = pd.read_excel(PATH_DATA+'pibpc.xls', sheet_name = 'pibpc')

In [ ]:
pibpc.head()

## Construcción del dataset <a class="anchor" id="02"></a>
[Tabla de Contenidos](#0)

El dataset recomendacion_finales es el resultado de la Tarea 3 Recomendación. Contiene el pk_cid de los 10.000 clientes a los cuales se les enviarán un email, la recomendación, precio del producto a recomendar y probabilidad de la compra. Partiremos de este dataset para construir el dataset de la segmentación para la personalización.

In [ ]:
recomendaciones_finales.head()

In [ ]:
personalizacion=recomendaciones_finales.copy(deep=True)

Añadiremos el resto de ficheros al dataset de personalización escogiendo solo los datos de la partición correspondiente al último mes disponible

In [ ]:
comercial=comercial[comercial['pk_partition']=='2019-05-28']
comercial.drop(columns=["pk_partition"], inplace=True)
socio=socio[socio['pk_partition']=='2019-05-28']
socio.drop(columns=["pk_partition"], inplace=True)

In [ ]:
personalizacion=pd.merge(personalizacion, socio, on=['pk_cid'], how='left') 

In [ ]:
personalizacion=pd.merge(personalizacion, comercial, on=['pk_cid'], how='left') 

In [ ]:
# corrigiendo la fecha de entry_date mal codificada
personalizacion.loc[personalizacion['entry_date'] == '2015-02-29', 'entry_date'] = '2015-02-28'
personalizacion.loc[personalizacion['entry_date'] == '2019-02-29', 'entry_date'] = '2019-02-28'
personalizacion["fecha_entrada"] = pd.to_datetime(personalizacion["entry_date"], format = "%Y-%m-%d")

In [ ]:
personalizacion.head()

### Incorporación, generación y transformación de atributos <a class="anchor" id="03"></a>
[Tabla de Contenido](#0)

Incluimos el atributo PIB per cápita y población por provincia y por países.
También incluimos si en el país o provincia se habla otro idioma.
Para ello, construimos una variable 'residencia_id' que para España toma region_code y para fuera de España coge country_id. Así, tenemos la población, PIB per cápita y el idioma referidos a España a nivel de provincia y para el resto de países a nivel de capitales de países.

In [ ]:
personalizacion['residencia_id']=personalizacion['country_id']
personalizacion.replace({'residencia_id' : 'ES'}, np.nan, inplace=True)
nan_filter = pd.Series(personalizacion['residencia_id'][personalizacion['country_id'] =='ES'].isnull())
nan_filter = nan_filter[nan_filter].index.values
nan_filter= pd.Series(nan_filter) 
personalizacion.loc[nan_filter, 'residencia_id'] = personalizacion.loc[nan_filter, 'residencia_id'].fillna(personalizacion['region_code'])

In [ ]:
personalizacion=pd.merge(personalizacion, pibpc, on=['residencia_id'], how='left')

In [ ]:
personalizacion.head()

Del dataset de productos nos gustaria tener la media de productos que tiene mensualmente cada uno de los clientes. 

In [ ]:
productos["payroll"].fillna(0, inplace=True)
productos["pension_plan"].fillna(0, inplace=True)
productos['pension_plan']=productos['pension_plan'].astype(int)
productos['payroll']=productos['pension_plan'].astype(int)

In [ ]:
productos.tail()

In [ ]:
productos['total'] = productos.iloc[:,2:].sum(axis=1)

In [ ]:
media_producto=productos[['pk_cid', 'total']]
media_productos=media_producto.groupby(['pk_cid']).mean()[['total']].reset_index()
media_productos

In [ ]:
# incorporamos la cantidad media mensual de productos que tienen los clientes
personalizacion=pd.merge(personalizacion, media_productos, on=['pk_cid'], how='left')

In [ ]:
personalizacion.head()

In [ ]:
personalizacion.rename(columns={"total":"media_prod"}, inplace=True)

In [ ]:
# calculamos la antiguedad del cliente en meses para incorporarla como nuevo atributo
personalizacion["ultimo_mes"]="2019-05-28"
personalizacion["ultimo_mes"] = pd.to_datetime(personalizacion["ultimo_mes"], format = "%Y-%m-%d")
personalizacion["antiguedad"]=(personalizacion["ultimo_mes"]-personalizacion["fecha_entrada"])/np.timedelta64(1,'M')

In [ ]:
personalizacion.info()

In [ ]:
# Dummy del gender con varon==1 y mujer==0
personalizacion = pd.get_dummies(personalizacion, columns = ["gender"], drop_first = True)

### Limpieza de atributos <a class="anchor" id="04"></a>
[Tabla de Contenido](#0)

Borramos las columnas que son object, id que ya no necesitamos. También borramos las columnas que no aportarán información para una comunicación distinta entre un cluster y otra como, por ejemplo, entry_channel (al no tener información sobre estos canales de entrada de los clientes, no podemos saber si se distinguen entre un canal y otro)

In [ ]:
# verificamos que no hay fallecidos en el dataset para poder eliminar esta columna
personalizacion['deceased'].value_counts()

In [ ]:
# se borra active_customer porque solo hay 7 clientes que no son activos en la plataforma en los últimos 3 meses
personalizacion['active_customer'].value_counts()

In [ ]:
col_borrar=['recomendacion', 'precio', 'prob', 'country_id', 'region_code','deceased', 'entry_channel', 'residencia_id', 'entry_date', 'fecha_entrada', 'ultimo_mes', 'active_customer']
personalizacion.drop(col_borrar, axis=1, inplace=True)

PIBpc y Población nos pueden indicar el tamaño y caracteristica del lugar de residencia del cliente, observamos que entre PIBpc y población existe una alta correlación, por lo que nos quedaremos únicamente con la población

In [ ]:
personalizacion.corr().style.background_gradient(cmap='coolwarm').set_precision(3)

In [ ]:
personalizacion.drop(['pibpc'], axis=1, inplace=True)

### Tratamiento de nulos <a class="anchor" id="05"></a>
[Tabla de Contenido](#0)

In [ ]:
personalizacion.info()

In [ ]:
personalizacion.describe()

In [ ]:
personalizacion.isnull().sum()

Tenemos 2 nulos en segment y 3993 en salary. 

Primero, vemos cuáles son esos 2 nulos en segment. Como no vemos ninguna característica llamativa que nos haga pensar diferente (y por el poco impacto que tendrá en el modelo), se asignan a la categoría de segment más frecuente

In [ ]:
personalizacion[personalizacion["segment"].isnull()]

In [ ]:
personalizacion["segment"].fillna("02 - PARTICULARES", inplace=True)

Para el salario, vamos la relación entre el salario y segment, para ver si podemos asignar el salario en función del segment, tal y como, lo explicamos en la Tarea 3: Recomendación

In [ ]:
# primero creamos una función que nos permitirá explorar la relación entre las variables segment (Categórica) y salary (numérica)

def explore_cat_values(dataframe, column_cat, comparar_column):

    _results_df=dataframe.pivot_table(index= column_cat, values=comparar_column, aggfunc=[len, np.mean, np.median]).sort_values(by=[('len', comparar_column)], ascending=False)
    _results_df.columns=['Observaciones', 'Promedio', 'Mediana' ]

    plt.figure(figsize=(15,10))

    ax1 = plt.subplot(2,1,1)
    ## Graficamos el conteo de cada uno de los valores
    ax1 = sns.countplot(
        dataframe[column_cat], order = list(dataframe[column_cat].value_counts().index)
    )
    ax2 = plt.subplot(2,1,2) # share ax1 para que me pinte el axis en el mismo orden

    ax2 = sns.barplot(
        data = dataframe, 
        x = column_cat,
        y = comparar_column,
        order = list(dataframe[column_cat].value_counts().index)
    )
    return _results_df
    plt.show()

In [ ]:
explore_cat_values(personalizacion, 'segment', 'salary')

Vemos una relación entre segment y salary (ingresos brutos de la unidad familiar) que podría tener sentido: 
Los 01- TOP con un salario más alto. 
02- PARTICULARES con un salario más bajo podría incluir desempleados, autónomos y trabajadores por cuenta ajena, por lo tanto, podría haber una mayor variabilidad de salarios y algunos bajos.
03- UNIVERSITARIOS con un salario intermedio podría incluir estudiantes universitarios que, aunque no estuviesen trabajando, consideran los ingresos de la unidad familiar y universitarios que podrían estar trabajando.

In [ ]:
explore_cat_values(personalizacion, 'segment', 'age')

Al observar la relación entre segment y edad parece también tener sentido: Los top con un promedio de edad de 50 años son los mayores, los universitarios con una edad más baja y los particulares con una edad promedio comprendida entre estos dos segmentos. Por tanto, validamos el atributo segment y lo utilizaremos para rellenar los nulos de los salarios.

Los nulos de los salarios lo rellenaremos en función de la mediana del salario de cada segmento puesto que la mediana es menos sensible que el promedio a los valores extremos. 

In [ ]:
personalizacion[personalizacion["segment"]=="01 - TOP"]['salary'].median()

In [ ]:
# para los del segment 01 -TOP
nan_filter = pd.Series(personalizacion['salary'][personalizacion['segment'] =='01 - TOP'].isnull())
nan_filter = nan_filter[nan_filter].index.values
nan_filter= pd.Series(nan_filter)
personalizacion.loc[nan_filter, 'salary'] = personalizacion.loc[nan_filter, 'salary'].fillna(personalizacion[personalizacion["segment"]=="01 - TOP"]['salary'].median())

In [ ]:
personalizacion[personalizacion["segment"]=="02 - PARTICULARES"]['salary'].median()

In [ ]:
# Asignamos del salario para el segment 02 - PARTICULARES
nan_filter = pd.Series(personalizacion['salary'][personalizacion['segment'] =='02 - PARTICULARES'].isnull())
nan_filter = nan_filter[nan_filter].index.values
nan_filter= pd.Series(nan_filter)
personalizacion.loc[nan_filter, 'salary'] = personalizacion.loc[nan_filter, 'salary'].fillna(personalizacion[personalizacion["segment"]=="02 - PARTICULARES"]['salary'].median())

In [ ]:
personalizacion[personalizacion["segment"]=="03 - UNIVERSITARIO"]['salary'].median()

In [ ]:
# para los del segment 03 - UNIVERSITARIO
nan_filter = pd.Series(personalizacion['salary'][personalizacion['segment'] =='03 - UNIVERSITARIO'].isnull())
nan_filter = nan_filter[nan_filter].index.values
nan_filter= pd.Series(nan_filter)
personalizacion.loc[nan_filter, 'salary'] = personalizacion.loc[nan_filter, 'salary'].fillna(personalizacion[personalizacion["segment"]=="03 - UNIVERSITARIO"]['salary'].median())

In [ ]:
# Eliminamos segment del dataset, pues no la utilizaremos para el clustering
personalizacion.drop(['segment'], axis=1, inplace=True)

In [ ]:
# comprobamos que ya no tenemos nulos
personalizacion.isnull().sum()

## Selección de los datos <a class="anchor" id="06"></a>
[Tabla de Contenido](#0)

In [ ]:
personalizacion.info()

In [ ]:
personalizacion.describe()

In [ ]:
# asignamos 'pk_cid' como indice
personalizacion.set_index('pk_cid', inplace=True) 

# Modelling <a class="anchor" id="07"></a>
[Tabla de Contenido](#0)

Como explicamos anteriormente, usaremos el algoritmo de K-Means para segmentar los 10.000 clientes

In [ ]:
# Eliminación de outliers porque pueden distorsionar los resultados del KMeans 
# (después de calcular los centroides, los incorporaremos para asignarlos a un cluster).

# Los atributos con mayores outliers son salary y age, por lo que consideraremos los que están hasta el quantile .90. 

# creamos los booleanos donde se cumplen que los clientes no son outliers en alguna de las columnas
criteria1 = personalizacion["salary"] < np.quantile(personalizacion["salary"], q = 0.90)
criteria2 = personalizacion["age"] < np.quantile(personalizacion["age"], q = 0.90)

# chained operations: juntamos los dos criterios. Por tanto nos quedaremos sólo con los clientes que no son outliers
# en salary ni en edad
criteria_final = criteria1 & criteria2
personalizacion_sin_out = personalizacion[criteria_final]

In [ ]:
# hemos excluidos casi 2000 casos aproximadamente.
criteria_final.value_counts()

In [ ]:
personalizacion_sin_out = personalizacion[(personalizacion["salary"] < np.quantile(personalizacion["salary"], q = 0.90)) & (personalizacion["age"] < np.quantile(personalizacion["age"], q = 0.90))]

In [ ]:
# El algoritmo requiere que escalemos los datos, por lo que utilizamos Standard Scaler
standard_scaler = StandardScaler()
scaled_df = standard_scaler.fit_transform(personalizacion_sin_out)
scaled_df = pd.DataFrame(scaled_df, index = personalizacion_sin_out.index, columns = personalizacion_sin_out.columns)

### Elbow curve <a class="anchor" id="08"></a>
[Tabla de Contenido](#0)

Para definir el número de clusters óptimo utilizaremos el Elbow Curve

In [ ]:
CALCULATE_ELBOW = True

if CALCULATE_ELBOW:
    st = time.time()

    sse = {}

    for k in range(2, 10):

        print(f"Fitting pipe with {k} clusters")
        cluster_model = KMeans(n_clusters = k)
        cluster_model.fit(scaled_df)

        sse[k] = cluster_model.inertia_

    et = time.time()
    print("Elbow curve took {} minutes.".format(round((et - st)/60), 2))

In [ ]:
if CALCULATE_ELBOW:
    fig = plt.figure(figsize = (16, 8))
    ax = fig.add_subplot()

    x_values = list(sse.keys())
    y_values = list(sse.values())

    ax.plot(x_values, y_values, label = "Inertia/dispersión de los clústers")
    fig.suptitle("Variación de la dispersión de los clústers en función de la k", fontsize = 16);

El mayor cambio de pendiente en la curva del codo se da entre el cluster 5.

In [ ]:
cluster_model = KMeans(n_clusters = 5)
cluster_model.fit(scaled_df)

In [ ]:
# se genera el dataframe escalado e incorporando los outliers.
scaled_df_with_outliers = standard_scaler.transform(personalizacion)

In [ ]:
scaled_df_with_outliers = pd.DataFrame(scaled_df_with_outliers, 
                                       index = personalizacion.index, 
                                       columns = personalizacion.columns)

In [ ]:
scaled_df_with_outliers.head()

In [ ]:
# calculamos el cluster de cada cliente, a partir del dataframe escalado y con outliers
labels = cluster_model.predict(scaled_df_with_outliers)

In [ ]:
personalizacion["cluster"] = labels

In [ ]:
personalizacion.shape

In [ ]:
selected_columns = ['age', 'salary', 'idiomas', 'media_prod', 'poblacion', 'gender_V', 'antiguedad']

sns.pairplot(personalizacion, vars = selected_columns, hue = 'cluster');

In [ ]:
# Vemos el tamaño de cada cluster
personalizacion.groupby("cluster").size()

In [ ]:
personalizacion ['cluster'] =personalizacion ['cluster'] + 1

In [ ]:
columnas = list(personalizacion.columns)
columnas.remove('cluster')

ficha_cluster= pd.pivot_table( personalizacion, index='cluster', values = columnas, aggfunc='mean')

ficha_cluster[columnas].style.background_gradient(cmap='coolwarm').set_precision(3)

### Descripción de los clusters <a class="anchor" id="09"></a> 
[Tabla de Contenido](#0)


· Cluster 1 - Urbanitas VIP: Clientes que viven en grandes ciudades con un alto salario. Se recomienda una comunicación formal y profesional, en castellano. En relación con el género, la comunicación deberá ser neutra, pues la componen tanto hombres como mujeres.

· Cluster 2 - Bilingues: Este cluster está compuesto por clientes que hablan otro idioma como inglés, catalán o vasco, por lo que se recomienda hacer la comunicación en dos idiomas usando un tono semiformal y neutro en género.

· Cluster 3 y 4 - Hombres y Mujeres en ciudades pequeñas: En el cluster 3 hay solo hombres y en el cluster 4 solo mujeres, por tanto, la comunicación deberá dirigirse en el cluster 3 en masculino y en el cluster 4 en femenino. Para ambos clusters se recomienda que el tono de la comunicación sea amigable, puesto que son personas que viven en ciudades pequeñas y que, normalmente, las relaciones con los clientes suelen ser más cercanas y menos formales. 


· Cluster 5 - Clientes leales y satisfechos: Son clientes que tienen varios de nuestros productos y han estado contratando nuestros servicios desde hace tiempo, por lo que se recomienda escribir unas líneas para agradecerles el tiempo que han estado con nosotros como clientes y la confianza en nuestra empresa y productos. La comunicación deberá ser en castellano, neutro en género y en tono semiformal, puesto que viven en ciudades de tamaño mediano y son personas adultas en general. 

# Conclusiones <a class="anchor" id="10"></a>
[Tabla de Contenido](#0)

En la tarea 4 de personalización de los correos que recibirán 10.000 clientes de la campaña de recomendación de los mejores productos, hemos utilizado el algoritmo de K-Means para agrupar a los clientes en función de ciertas características que nos ayudarán a dirigirnos a ellos de forma más efectiva.

Encontramos que lo óptimo sería agrupar a los clientes en 5 segmentos y diferenciar el escrito de la comunicación en función de la edad, salario, idioma, lugar de residencia del cliente, antiguedad como cliente, cantidad de productos contratados y sexo. 